ЗАДАНИЕ

Напишите декоратор, оптимизирующий работу декорируемой функции. Декоратор должен сохранять результат работы функции на ближайшие три запуска и вместо выполнения функции возвращать сохранённый результат. 
После трёх запусков функция должна вызываться вновь, а результат работы функции — вновь кешироваться.

Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

РЕШЕНИЕ
1. Импорт библиотек

requests - библиотека для выполнения HTTP-запросов.

os - библиотека для взаимодействия с операционной системой, хотя в этом коде она не используется.

time - библиотека для работы со временем, используется для измерения производительности.

ThreadPoolExecutor и ProcessPoolExecutor - классы из модуля concurrent.futures позволяют выполнять функции асинхронно, либо в потоках, либо в процессах.

In [ ]:
import requests
import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

2. Декоратор cache_results

Назначение: Декоратор, который кэширует результаты функции, чтобы не выполнять её повторно для одних и тех же аргументов больше трёх раз.
cache - cловарь для хранения результатов вызовов функции.
call_count - cчетчик количества вызовов функции.
wrapper: Вложенная функция, которая оборачивает оригинальную функцию. Она проверяет, была ли функция вызвана с теми же аргументами, и если да, возвращает закэшированный результат. Если функция вызывается более трех раз, кэш очищается.

In [ ]:
def cache_results(func):
    cache = {}
    call_count = 0

    def wrapper(*args):
        nonlocal call_count
        call_count += 1
        
        if call_count <= 3 and args in cache:
            return cache[args]
        
        result = func(*args)
        if call_count <= 3:
            cache[args] = result
        else:
            call_count = 1
            cache.clear()  # Очищаем кеш, если было больше 3 вызовов
        
        return result

    return wrapper

3. Функция download_file

Назначение: Загружает файл по указанному URL и сохраняет его на локальном диске.
Имя файла: Генерируется из URL, заменяя недопустимые символы.
requests.get - отправляет HTTP GET-запрос по указанному URL.
r.raise_for_status() - проверяет, был ли запрос успешным (статус-код 200). Если нет, выбрасывает исключение.
Запись файла: Файл записывается в бинарном режиме, используя потоковое чтение данных для экономии памяти.

In [ ]:
@cache_results
def download_file(url):
    # Генерируем имя файла, убирая недопустимые символы
    local_filename = url.split('/')[-1].replace('?', '_')  # Заменяем '?' на '_'
    
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

4. Функция download_files_in_threads

Назначение: Загружает файлы по списку URL-адресов, используя многопоточность.
ThreadPoolExecutor: Создает пул потоков, позволяющий одновременно выполнять несколько загрузок.
executor.map: Применяет функцию download_file ко всем URL в списке urls.


In [ ]:
def download_files_in_threads(urls):
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_file, urls)

5. Функция download_files_in_processes

Назначение: Загружает файлы по списку URL-адресов, используя многопроцессорность.
ProcessPoolExecutor: Создает пул процессов, позволяя выполнять загрузки в разных процессах, что может быть более эффективно для CPU-bound задач.

In [ ]:
def download_files_in_processes(urls):
    with ProcessPoolExecutor(max_workers=5) as executor:
        executor.map(download_file, urls)

6. Функция generate_file_urls

Назначение: Генерирует список из 100 URL-адресов для изображений размером 150x150 пикселей с текстом от 0 до 99.
Возврат: Возвращает список строк, представляющих URL-адреса.

In [ ]:
def generate_file_urls():
    # Генерация списка URL-адресов
    return [f"https://via.placeholder.com/150?text={i}" for i in range(100)]

7. Основной блок кода

Назначение: Основной блок, который выполняется при запуске скрипта.
Генерация URL: Создает список URL-адресов с помощью generate_file_urls().
Измерение времени: Сравнивает время загрузки с использованием трех различных методов (последовательный, многопоточный и многопроцессорный) и выводит результаты.

In [ ]:
if __name__ == "__main__":
    urls = generate_file_urls()

    # Сравнение производительности последовательного метода
    start_time = time.time()
    for url in urls:
        download_file(url)
    sequential_time = time.time() - start_time
    print(f"Sequential download time: {sequential_time:.2f} seconds")

    # Сравнение производительности с использованием потоков
    start_time = time.time()
    download_files_in_threads(urls)
    thread_time = time.time() - start_time
    print(f"Threaded download time: {thread_time:.2f} seconds")

    # Сравнение производительности с использованием процессов
    start_time = time.time()
    download_files_in_processes(urls)
    process_time = time.time() - start_time
    print(f"Process download time: {process_time:.2f} seconds")

ВЫВОД:

На выходе программы мы получаем что-то такое:

In [ ]:
Sequential download time: X.XX seconds
Threaded download time: Y.YY seconds
Process download time: Z.ZZ seconds

X.XX — время, затраченное на последовательную загрузку всех 100 изображений.

Y.YY — время, затраченное на загрузку изображений с использованием потоков.

Z.ZZ — время, затраченное на загрузку изображений с использованием процессов.